In [9]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import Statevector,Operator,SparsePauliOp
from qiskit.primitives import StatevectorSampler,StatevectorEstimator
from scipy.optimize import minimize
import numpy as np
import matplotlib.pyplot as plt

from qiskit_addon_cutting.utils.observable_grouping import ObservableCollection
qubits=3
def generate_covariance_band_decay_matrix(n, bandwidth, decay_rate):
    """
    Generate a real symmetric positive semidefinite (PSD) covariance matrix
    with decaying values away from the diagonal.

    Parameters:
        n (int): Size of the matrix (n x n).
        bandwidth (int): Controls effective band width (must be > 0).
        decay_rate (float): Exponential decay factor.

    Returns:
        np.ndarray: Covariance matrix (real symmetric PSD).
    """
    if bandwidth <= 0:
        raise ValueError("Bandwidth must be positive.")

    A = np.zeros((n, n))

    for i in range(n):
        for j in range(n):  # no need to restrict to upper triangle
            distance = abs(i - j)
            decay = np.exp(-decay_rate * (distance / bandwidth) ** 2)
            A[i, j] = decay * np.random.rand()

    # Make it PSD
    C = A @ A.T

    # Optional: ensure numerical PSD
    eigvals, eigvecs = np.linalg.eigh(C)
    eigvals_clipped = np.clip(eigvals, 0, None)
    C = eigvecs @ np.diag(eigvals_clipped) @ eigvecs.T

    return C



hermitian_matrix =generate_covariance_band_decay_matrix(n=2**qubits, bandwidth=80, decay_rate=120)
print(f"Hermitian Matrix:{hermitian_matrix}")
z=Operator(-hermitian_matrix)
pauli_op=SparsePauliOp.from_operator(z)
print(pauli_op)

Hermitian Matrix:[[0.81390617 0.70726189 0.47690835 1.38765579 1.06010497 1.0437334
  1.14773486 0.67604905]
 [0.70726189 1.02273539 0.90507855 1.7552227  1.33655469 1.24716887
  1.37143088 0.86255749]
 [0.47690835 0.90507855 1.68621198 1.8671163  1.36412838 0.91325743
  1.35858022 0.95118803]
 [1.38765579 1.7552227  1.8671163  3.79768263 2.810139   1.9205662
  2.65479021 1.45040738]
 [1.06010497 1.33655469 1.36412838 2.810139   2.16576399 1.54636535
  1.98796065 1.16183179]
 [1.0437334  1.24716887 0.91325743 1.9205662  1.54636535 1.98279843
  1.74832861 1.25621127]
 [1.14773486 1.37143088 1.35858022 2.65479021 1.98796065 1.74832861
  2.37859545 1.21748707]
 [0.67604905 0.86255749 0.95118803 1.45040738 1.16183179 1.25621127
  1.21748707 0.93661028]]
SparsePauliOp(['III', 'IIX', 'IIZ', 'IXI', 'IXX', 'IXZ', 'IYY', 'IZI', 'IZX', 'IZZ', 'XII', 'XIX', 'XIZ', 'XXI', 'XXX', 'XXZ', 'XYY', 'XZI', 'XZX', 'XZZ', 'YIY', 'YXY', 'YYI', 'YYX', 'YYZ', 'YZY', 'ZII', 'ZIX', 'ZIZ', 'ZXI', 'ZXX', 'ZXZ', '

In [10]:
# Step 3: Group Pauli terms using ObservableCollection
collection = ObservableCollection(pauli_op.paulis)

print(f"\nTotal QWC Groups formed: {len(collection.groups)}")

# Step 4: Show group contents
for idx, group in enumerate(collection.groups):
    print(f"\nGroup {idx + 1}:")
    print("  Measurement basis (general observable):", group.general_observable)
    print("  Commuting Pauli terms:")
    for term in group.commuting_observables:
        print("   ", term)


Total QWC Groups formed: 14

Group 1:
  Measurement basis (general observable): XYY
  Commuting Pauli terms:
    XYY
    IYY
    XII
    III

Group 2:
  Measurement basis (general observable): YXY
  Commuting Pauli terms:
    YXY
    YIY
    IXI

Group 3:
  Measurement basis (general observable): YYX
  Commuting Pauli terms:
    YYX
    YYI
    IIX

Group 4:
  Measurement basis (general observable): YYZ
  Commuting Pauli terms:
    YYZ
    IIZ

Group 5:
  Measurement basis (general observable): YZY
  Commuting Pauli terms:
    YZY
    IZI

Group 6:
  Measurement basis (general observable): ZYY
  Commuting Pauli terms:
    ZYY
    ZII

Group 7:
  Measurement basis (general observable): XXX
  Commuting Pauli terms:
    XXX
    IXX
    XIX
    XXI

Group 8:
  Measurement basis (general observable): XXZ
  Commuting Pauli terms:
    XXZ
    IXZ
    XIZ

Group 9:
  Measurement basis (general observable): XZX
  Commuting Pauli terms:
    XZX
    IZX
    XZI

Group 10:
  Measurement basis (ge

In [11]:
# Step 5: Print grouping advantage metric
num_terms = len(pauli_op)
num_groups = len(collection.groups)
efficiency = num_terms / num_groups
reduction_ratio = (1 - (num_groups / num_terms)) * 100

print("\n" + "="*50)
print("📊 Measurement Optimization Summary")
print(f"🔢 Total Pauli terms: {num_terms}")
print(f"📦 Total QWC groups (unique basis rotations needed): {num_groups}")
print(f"⚙️ Grouping Efficiency (terms per measurement basis): {efficiency:.2f}")
print(f"📉 Reduction in circuit count: {reduction_ratio:.2f}%")
print("="*50)



📊 Measurement Optimization Summary
🔢 Total Pauli terms: 36
📦 Total QWC groups (unique basis rotations needed): 14
⚙️ Grouping Efficiency (terms per measurement basis): 2.57
📉 Reduction in circuit count: 61.11%


In [84]:
import numpy as np
from qiskit.quantum_info import Operator, SparsePauliOp
from scipy.linalg import eigh

# Parameters
qubits = 6
dim = 2**qubits

def generate_covariance_band_decay_matrix(n, bandwidth, decay_rate):
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            distance = abs(i - j)
            decay = np.exp(-decay_rate * (distance / bandwidth) ** 2)
            A[i, j] = decay * np.random.rand()
    C = A @ A.T
    eigvals, eigvecs = np.linalg.eigh(C)
    eigvals_clipped = np.clip(eigvals, 0, None)
    return eigvecs @ np.diag(eigvals_clipped) @ eigvecs.T

# Step 1: Generate the matrix
H_matrix = generate_covariance_band_decay_matrix(dim, bandwidth=10, decay_rate=100)

# Step 2: Convert to Pauli operator
op = Operator(H_matrix)
pauli_op = SparsePauliOp.from_operator(op)

# Step 3: Filter local Pauli terms
def is_local(pauli_str, max_distance=1):
    indices = [i for i, p in enumerate(pauli_str) if p != 'I']
    if len(indices) <= 1:
        return True
    return max(indices) - min(indices) <= max_distance

filtered_labels = []
filtered_coeffs = []

for coeff, label in zip(pauli_op.coeffs, pauli_op.paulis.to_labels()):
    if is_local(label, max_distance=3):  # Nearest-neighbor locality
        filtered_labels.append(label)
        filtered_coeffs.append(coeff)

# Step 4: Rebuild operator
filtered_op = SparsePauliOp.from_list(list(zip(filtered_labels, filtered_coeffs)))
filtered_matrix = filtered_op.to_operator().data

# Step 5: Compute eigenvalues
eigvals_full = np.linalg.eigvalsh(H_matrix)
eigvals_local = np.linalg.eigvalsh(filtered_matrix)

print("🔹 Full Hamiltonian Eigenvalues:")
print(np.round(eigvals_full, 4))

print("\n🔹 Local (banded) Pauli Hamiltonian Eigenvalues:")
print(np.round(eigvals_local, 4))


🔹 Full Hamiltonian Eigenvalues:
[0.0000e+00 0.0000e+00 3.0000e-04 2.2000e-03 2.3000e-03 3.2000e-03
 4.0000e-03 5.4000e-03 9.0000e-03 9.9000e-03 1.1800e-02 1.3100e-02
 1.6900e-02 2.1600e-02 2.4400e-02 3.2700e-02 3.7000e-02 3.7900e-02
 4.0200e-02 4.0200e-02 4.5200e-02 6.0500e-02 6.2200e-02 6.8200e-02
 7.9600e-02 9.9800e-02 1.0840e-01 1.3950e-01 1.4040e-01 1.4620e-01
 1.6600e-01 1.9820e-01 2.0290e-01 2.1830e-01 2.5740e-01 2.6770e-01
 3.1200e-01 3.1540e-01 3.6250e-01 3.6320e-01 3.7460e-01 4.3860e-01
 4.4190e-01 4.8160e-01 5.1810e-01 5.8290e-01 5.8960e-01 5.9810e-01
 6.1240e-01 6.6930e-01 7.4980e-01 8.2610e-01 8.4930e-01 9.0300e-01
 9.1440e-01 1.0140e+00 1.0345e+00 1.0929e+00 1.1408e+00 1.1577e+00
 1.1898e+00 1.2220e+00 1.4680e+00 1.5126e+00]

🔹 Local (banded) Pauli Hamiltonian Eigenvalues:
[-0.1286 -0.122  -0.0958 -0.0927 -0.0827 -0.0769 -0.0441 -0.0341 -0.0226
 -0.0142 -0.0026  0.0296  0.0343  0.0449  0.048   0.0501  0.0715  0.1104
  0.1193  0.1338  0.1476  0.1597  0.1836  0.1852  0.2022 

In [82]:
print(H_matrix[4,:])

[ 0.0387472   0.05918397  0.05436857  0.07559324  0.10344732  0.07850105
  0.1269582   0.11571719  0.09228152  0.10114482  0.14813036  0.127804
  0.15505313  0.11705022  0.09442211  0.1621275   0.09639157  0.12549284
  0.09346767  0.07730248  0.07890047  0.07964286  0.06178708  0.06561366
  0.07776682  0.04209264  0.04149097  0.03362397  0.0261969   0.02203023
  0.01408724  0.00860076  0.00239685  0.00263117 -0.0034289  -0.00604527
 -0.01177745 -0.01115098 -0.01822046 -0.02327586 -0.0227512  -0.02680788
 -0.02030573 -0.0223137  -0.02265921 -0.01158455 -0.00445398  0.00124928
  0.00490937  0.01164737  0.01425313  0.01990438  0.02134604  0.0317643
  0.03652078  0.04052079  0.02668705  0.05266229  0.03511972  0.03308816
  0.02978999  0.0256772   0.02509196  0.02023899]
